In [ ]:
from google.colab import drive
import os
import sys

drive.mount("/content/drive")

# install dependencies:
!pip install pyyaml==5.1

import torch
import torchvision

print(torch.__version__, torch.cuda.is_available())
!gcc --version
# install detectron2: (colab has CUDA 10.1 + torch 1.8)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
assert torch.__version__.startswith(
    "1.8"
)  # please manually install torch 1.8 if Colab changes its default version
!pip install detectron2==0.4 -q -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
# Colab's "restart runtime" is needed after installation
# !pip install -q -U albumentations   # Check if still needed

os.chdir("drive/MyDrive/PG/pg_iss/")
assert os.getcwd().split("/")[-1] == "pg_iss"
# clone the repo at the same version, in order to access pre-defined configs
# !git clone --branch v0.4 https://github.com/facebookresearch/detectron2.git detectron2_repo
!pip install -q -e detectron2_repo/projects/TensorMask/

In [ ]:
%load_ext autoreload
%autoreload 2

from detectron2.utils import comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.engine import (
    DefaultTrainer,
    DefaultPredictor,
    default_argument_parser,
    default_setup,
    launch,
)
from detectron2.evaluation import COCOEvaluator, verify_results
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog
import random, cv2
import numpy as np
from google.colab.patches import cv2_imshow

sys.path.append("detectron2_repo/projects/TensorMask")

from lib import coco_handler, copy_and_paste_augm, constants
from lib.copy_and_paste_detectron import *

from tensormask import add_tensormask_config
import tensormask

In [ ]:
cfg = get_cfg()
add_tensormask_config(cfg)
add_cap_config(cfg)
cfg.merge_from_file("configs/CAP_config.yaml")
# print(cfg.dump())

In [ ]:
# currently Trainer can not be build with an existing dataset, hence this is waste RAM
show_demo = False
if show_demo:
    dataset = CapDataset(cfg)
    for i in range(10):
        CapDataset.show_image_and_mask(dataset[i])
        dataset.visualize_pools()

In [ ]:
trainer = Trainer(cfg)  # takes some time for creating object pools
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
register_coco_instances(
    "boxes", {}, "data/raw/annotations/all_anno.json", "data/raw/imgs"
)
# TODO also test on some generated images
# TODO keep track of instances and only count instances that weren't used for training
dataset_dicts = DatasetCatalog.get("boxes")

In [ ]:
from detectron2.utils.visualizer import ColorMode, Visualizer
import random, cv2

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.TENSOR_MASK.SCORE_THRESH_TEST = (
    0.5  # only show prediction with high confidence
)
predictor = DefaultPredictor(cfg)

for d in dataset_dicts:
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(
        im[:, :, ::-1],
        metadata=MetadataCatalog.get("boxes"),
        scale=0.8,
        instance_mode=ColorMode.IMAGE_BW,  # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])